### Imports

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm import tqdm
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

%run updatePreprocessing.ipynb

[['ق', 'و', 'ل', 'ه'], ['ل', 'ع', 'د', 'م'], ['م', 'ا'], ['ت', 'ت', 'ع', 'ل', 'ق'], ['إ', 'ل', 'خ'], ['أ', 'ي'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['ق', 'و', 'ل', 'ه'], ['م', 'ا'], ['م', 'ر'], ['أ', 'ي'], ['ق', 'ب', 'ي', 'ل'], ['ق', 'و', 'ل'], ['ا', 'ل', 'م', 'ت', 'ن'], ['ل', 'غ', 'ت'], ['و', 'ل', 'و'], ['ا', 'ق', 'ت', 'ص', 'ر'], ['ع', 'ل', 'ى'], ['أ', 'و', 'ص', 'ي', 'ت'], ['ل', 'ه'], ['ب', 'ش', 'ا', 'ة'], ['أ', 'و'], ['أ', 'ع', 'ط', 'و', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ل', 'ا'], ['غ', 'ن', 'م'], ['ل', 'ه'], ['ع', 'ن', 'د'], ['ا', 'ل', 'م', 'و', 'ت'], ['ه', 'ل'], ['ت', 'ب', 'ط', 'ل'], ['ا', 'ل', 'و', 'ص', 'ي', 'ة'], ['أ', 'و'], ['ي', 'ش', 'ت', 'ر', 'ى'], ['ل', 'ه'], ['ش', 'ا', 'ة'], ['و', 'ي', 'ؤ', 'خ', 'ذ'], ['م', 'ن'], ['ق', 'و', 'ل', 'ه'], ['ا', 'ل', 'آ', 'ت', 'ي'], ['ك', 'م', 'ا'], ['ل', 'و'], ['ل', 'م'], ['ي', 'ق', 'ل'], ['م', 'ن'], ['م', 'ا', 'ل', 'ي'], ['و', 'ل', 'ا'], ['م', 'ن'], ['غ', 'ن', 'م', 'ي'], ['أ', 'ن', 'ه', 'ا'], ['ل', 'ا'], ['ت', 'ب', 'ط', 'ل'], ['و', 'ع', 'ب', 'ا', 'ر',

### Constants

In [2]:
EMBEDDING_DIM = 200
HIDDEN_SIZE = 512
NUM_LAYERS = 1
NUM_EPOCHS = 10
LEARNING_RATE = 0.001
BATCH_SIZE = 256
VOCAB_SIZE = len(basic_arabic_letters) + 1
LABELS_SIZE = len(DIACRITICS)


TRAIN_PATH = "../dataset/train.txt"
VAL_PATH = "../dataset/val.txt"
LSTM_PATH="./models/lstm.pth"
RNN_PATH="./models/rnn.pth"
CBHG_PATH="./models/cbhg.pth"

In [3]:
from typing import List

class Prenet(nn.Module):
    """
    A prenet is a collection of linear layers with dropout(0.5),
    and RELU activation function
    Args:
    config: the hyperparameters object
    in_dim (int): the input dim
    """

    def __init__(
        self, in_dim: int, prenet_depth: List[int] = [256, 128], dropout: int = 0.5
    ):
        """ Initializing the prenet module """
        super().__init__()
        in_sizes = [in_dim] + prenet_depth[:-1]
        self.layers = nn.ModuleList(
            [
                nn.Linear(in_size, out_size)
                for (in_size, out_size) in zip(in_sizes, prenet_depth)
            ]
        )
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs: torch.Tensor):
        """Calculate forward propagation
        Args:
        inputs (batch_size, seqLen): the inputs to the prenet, the input shapes could
        be different as it is being used in both encoder and decoder.
        Returns:
        Tensor: the output of  the forward propagation
        """
        for linear in self.layers:
            inputs = self.dropout(self.relu(linear(inputs)))
        return inputs
class Highway(nn.Module):
    """Highway Networks were developed by (Srivastava et al., 2015)
    to overcome the difficulty of training deep neural networks
    (https://arxiv.org/abs/1507.06228).
    Args:
    in_size (int): the input size
    out_size (int): the output size
    """

    def __init__(self, in_size, out_size):
        """
        Initializing Highway networks
        """
        super().__init__()
        self.H = nn.Linear(in_size, out_size)
        self.H.bias.data.zero_()
        self.T = nn.Linear(in_size, out_size)
        self.T.bias.data.fill_(-1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs: torch.Tensor):
        """Calculate forward propagation
        Args:
        inputs (Tensor):
        """
        H = self.relu(self.H(inputs))
        T = self.sigmoid(self.T(inputs))
        return H * T + inputs * (1.0 - T)
class BatchNormConv1d(nn.Module):
    """
    A nn.Conv1d followed by an optional activation function, and nn.BatchNorm1d
    """

    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        kernel_size: int,
        stride: int,
        padding: int,
        activation = None,
    ):
        super().__init__()
        self.conv1d = nn.Conv1d(
            in_dim,
            out_dim,
            kernel_size=kernel_size,
            stride=stride,
            padding=padding,
            bias=False,
        )
        self.bn = nn.BatchNorm1d(out_dim)
        self.activation = activation

    def forward(self, x):
        x = self.conv1d(x)
        if self.activation is not None:
            x = self.activation(x)
        return self.bn(x)
class CBHG(nn.Module):
    """The CBHG module (1-D Convolution Bank + Highway network + Bidirectional GRU)
    was proposed by (Lee et al., 2017, https://www.aclweb.org/anthology/Q17-1026)
    for a character-level NMT model.
    It was adapted by (Wang et al., 2017) for building the Tacotron.
    It is used in both the encoder and decoder  with different parameters.
    """

    
    def __init__(
        self,
        in_dim: int,
        out_dim: int,
        K: int,
        projections,
        highway_layers: int = 4,
    ):
        """Initializing the CBHG module
        Args:
        in_dim (int): the input size
        out_dim (int): the output size
        k (int): number of filters
        projections: A list of integers representing the output sizes of convolutional projections.
        highway_layers: Number of highway layers.
        """
        super().__init__()

        self.in_dim = in_dim
        self.out_dim = out_dim
        self.relu = nn.ReLU()
        #list of modules each one of them will be BatchNoemConv1 
        #and each of them has a kernel size ranging from 1 -> k
        self.conv1d_banks = nn.ModuleList(
            [
                BatchNormConv1d(
                    in_dim,
                    in_dim,
                    kernel_size=k,
                    stride=1,
                    padding=k // 2,
                    activation=self.relu,
                )
                for k in range(1, K + 1)
            ]
        )
        '''
        kernel_size: The size of the window over which the maximum value is computed. It specifies the size of the pooling operation.
        stride: The step size to slide the pooling window along the input. If not specified, it defaults to kernel_size.
        padding: Zero-padding added to both sides of the input. Padding helps to control the spatial dimensions of the output.
        '''
        self.max_pool1d = nn.MaxPool1d(kernel_size=2, stride=1, padding=1)

        in_sizes = [K * in_dim] + projections[:-1]
        activations = [self.relu] * (len(projections) - 1) + [None]
        self.conv1d_projections = nn.ModuleList(
            [
                BatchNormConv1d(
                    in_size, out_size, kernel_size=3, stride=1, padding=1, activation=ac
                )
                for (in_size, out_size, ac) in zip(in_sizes, projections, activations)
            ]
        )

        self.pre_highway = nn.Linear(projections[-1], in_dim, bias=False)
        self.highways = nn.ModuleList([Highway(in_dim, in_dim) for _ in range(4)])

        self.gru = nn.GRU(in_dim, out_dim, 1, batch_first=True, bidirectional=True)

    def forward(self, inputs, input_lengths=None):
        # (B, T_in, in_dim)
        x = inputs
        x = x.transpose(1, 2)
        T = x.size(-1)

        # (B, in_dim*K, T_in)
        # Concat conv1d bank outputs
        x = torch.cat([conv1d(x)[:, :, :T] for conv1d in self.conv1d_banks], dim=1)
        assert x.size(1) == self.in_dim * len(self.conv1d_banks)
        x = self.max_pool1d(x)[:, :, :T]

        for conv1d in self.conv1d_projections:
            x = conv1d(x)

        # (B, T_in, in_dim)
        # Back to the original shape
        x = x.transpose(1, 2)

        if x.size(-1) != self.in_dim:
            x = self.pre_highway(x)

        # Residual connection
        x += inputs
        for highway in self.highways:
            x = highway(x)

        if input_lengths is not None:
            x = nn.utils.rnn.pack_padded_sequence(x, input_lengths, batch_first=True)

        # (B, T_in, in_dim*2)
        self.gru.flatten_parameters()
        outputs, _ = self.gru(x)

        if input_lengths is not None:
            outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)

        return outputs
class CBHGModel(nn.Module):
    """CBHG model implementation as described in the paper:
     https://ieeexplore.ieee.org/document/9274427

    Args:
    inp_vocab_size (int): the number of the input symbols
    targ_vocab_size (int): the number of the target symbols (diacritics)
    embedding_dim (int): the embedding  size
    use_prenet (bool): whether to use prenet or not
    prenet_sizes (List[int]): the sizes of the prenet networks
    cbhg_gru_units (int): the number of units of the CBHG GRU, which is the last
    layer of the CBHG Model.
    cbhg_filters (int): number of filters used in the CBHG module
    cbhg_projections: projections used in the CBHG module

    Returns:
    diacritics Dict[str, Tensor]:
    """

    def __init__(
        self,
        inp_vocab_size: int,
        targ_vocab_size: int,
        embedding_dim: int = 512,
        use_prenet: bool = True,
        prenet_sizes = [512, 256],
        cbhg_gru_units: int = 512,
        cbhg_filters: int = 16,
        cbhg_projections = [128, 256],
        post_cbhg_layers_units = [256, 256],
        post_cbhg_use_batch_norm: bool = True
    ):
        super().__init__()
        self.use_prenet = use_prenet
        self.embedding = nn.Embedding(inp_vocab_size, embedding_dim)
        if self.use_prenet:
            self.prenet = Prenet(embedding_dim, prenet_depth=prenet_sizes)

        self.cbhg = CBHG(
            prenet_sizes[-1] if self.use_prenet else embedding_dim,
            cbhg_gru_units,
            K=cbhg_filters,
            projections=cbhg_projections,
        )

        layers = []
        post_cbhg_layers_units = [cbhg_gru_units] + post_cbhg_layers_units

        for i in range(1, len(post_cbhg_layers_units)):
            layers.append(
                nn.LSTM(
                    post_cbhg_layers_units[i - 1] * 2,
                    post_cbhg_layers_units[i],
                    bidirectional=True,
                    batch_first=True,
                )
            )
            if post_cbhg_use_batch_norm:
                layers.append(nn.BatchNorm1d(post_cbhg_layers_units[i] * 2))

        self.post_cbhg_layers = nn.ModuleList(layers)
        self.projections = nn.Linear(post_cbhg_layers_units[-1] * 2, targ_vocab_size)
        self.post_cbhg_layers_units = post_cbhg_layers_units
        self.post_cbhg_use_batch_norm = post_cbhg_use_batch_norm


    def forward(
        self,
        src: torch.Tensor,
        lengths = None,
        target = None,  # not required in this model
    ):
        """Compute forward propagation"""

        # src = [batch_size, src len]
        # lengths = [batch_size]
        # target = [batch_size, trg len]

        embedding_out = self.embedding(src)
        # embedding_out; [batch_size, src_len, embedding_dim]

        cbhg_input = embedding_out
        if self.use_prenet:
            cbhg_input = self.prenet(embedding_out)

            # cbhg_input = [batch_size, src_len, prenet_sizes[-1]]

        outputs = self.cbhg(cbhg_input, lengths)

        hn = torch.zeros((2, 2, 2))
        cn = torch.zeros((2, 2, 2))

        for i, layer in enumerate(self.post_cbhg_layers):
            if isinstance(layer, nn.BatchNorm1d):
                outputs = layer(outputs.permute(0, 2, 1))
                outputs = outputs.permute(0, 2, 1)
                continue
            if i > 0:
                outputs, (hn, cn) = layer(outputs, (hn, cn))
            else:
                outputs, (hn, cn) = layer(outputs)


        predictions = self.projections(outputs)

        # predictions = [batch_size, src len, targ_vocab_size]

        # output = {"diacritics": predictions}

        return predictions

### Train

In [5]:
def train(model, path, train_dataset, train_labels, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE):
    """
    This function implements the training logic
    Inputs:
    - model: the model to be trained
    - train_dataset: the training set
    - batch_size: integer represents the number of examples per step
    - epochs: integer represents the total number of epochs (full training pass)
    - learning_rate: the learning rate to be used by the optimizer
    """

    # Create the dataloader of the training set (make the shuffle=True)
    tensor_train_dataset = TensorDataset(train_dataset, train_labels)
    train_dataloader = DataLoader(tensor_train_dataset, batch_size=batch_size, shuffle=True)

    # Make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss()

    # Create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # GPU configuration
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()
    
    best_accuracy = 0.0
    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        
        for train_input, train_label in tqdm(train_dataloader):
            # Zero your gradients
            optimizer.zero_grad()

            # Move the train input to the device
            train_label = train_label.to(device)

            # Move the train label to the device
            train_input = train_input.to(device)

            # Do the forward pass
            output = model(train_input).float()

            # Loss calculation
            batch_loss = criterion(output.view(-1, output.shape[-1]), train_label.view(-1))

            # Append the batch loss to the total_loss_train
            total_loss_train += batch_loss.item()
            
            # Calculate the batch accuracy (just add the number of correct predictions)
            # Compare predicted diacritic with true diacritic and count correct predictions
            correct_predictions = (output.argmax(dim=2) == train_label)

            # Calculate accuracy for the current batch
            acc = correct_predictions.sum().item()
            total_acc_train += acc

            # Do the backward pass
            batch_loss.backward()

            # Update the weights with your optimizer
            optimizer.step()     
        
        # Calculate the epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # Calculate the accuracy
        epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0]))

        print(f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')
        
        if epoch_acc > best_accuracy:
            best_accuracy = epoch_acc
            torch.save(model.state_dict(), path)
            print(f'Saved the best model with accuracy: {best_accuracy} to {path}\n')


In [9]:
def readFile(path):
	sentences = []
	with open(path, 'r', encoding='utf-8') as file:
		for line in file:
			sentences.append(line.strip())

	return sentences

corpus=  readFile(TRAIN_PATH)

x_train = []
y_train = []

for sentence in corpus[:100]:
	# Clean each sentence in the corpus
	# Get the char list for each word in the sentence and its corresponding diacritics
	char_list, diacritics_list = separate_words_and_diacritics(sentence.strip())

	x_train.append(char_list)
	y_train.append(diacritics_list)

X_train_padded = [torch.tensor([char_to_index[char] for char in word]) for sentence in x_train for word in sentence]
X_train_padded = pad_sequence(X_train_padded, batch_first=True)

y_train_padded = [torch.tensor([diacritic_to_index[char] for char in word]) for sentence in y_train for word in sentence]
y_train_padded = pad_sequence(y_train_padded, batch_first=True)

In [7]:
model=CBHGModel(
            embedding_dim=EMBEDDING_DIM,
            inp_vocab_size=VOCAB_SIZE,
            targ_vocab_size=LABELS_SIZE,
            use_prenet=False,
            prenet_sizes=[512, 256],
            cbhg_gru_units=256,
            cbhg_filters=16,
            cbhg_projections=[128, 256],
            post_cbhg_layers_units=[256, 256],
            post_cbhg_use_batch_norm=True,
        )
print(model)

CBHGModel(
  (embedding): Embedding(37, 200)
  (cbhg): CBHG(
    (relu): ReLU()
    (conv1d_banks): ModuleList(
      (0): BatchNormConv1d(
        (conv1d): Conv1d(200, 200, kernel_size=(1,), stride=(1,), bias=False)
        (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (1): BatchNormConv1d(
        (conv1d): Conv1d(200, 200, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (2): BatchNormConv1d(
        (conv1d): Conv1d(200, 200, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
        (bn): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (3): BatchNormConv1d(
        (conv1d): Conv1d(200, 200, kernel_size=(4,), stride=(1,), padding=(2,), bias=False)
        (bn): BatchNorm1d(

In [10]:
# train(model, CBHG_PATH, X_train_padded, y_train_padded)

100%|██████████| 18/18 [00:23<00:00,  1.31s/it]


Epochs: 1 | Train Loss: 0.0017729900237346767             | Train Accuracy: 0.8619992724038967

Saved the best model with accuracy: 0.8619992724038967 to ./models/lstm.pth



100%|██████████| 18/18 [00:23<00:00,  1.32s/it]


Epochs: 2 | Train Loss: 0.0010672102985831565             | Train Accuracy: 0.9106673673147662

Saved the best model with accuracy: 0.9106673673147662 to ./models/lstm.pth



100%|██████████| 18/18 [00:23<00:00,  1.30s/it]


...

Epochs: 8 | Train Loss: 0.00032923236926591043             | Train Accuracy: 0.9711184768988237

Saved the best model with accuracy: 0.9711184768988237 to ./models/lstm.pth



100%|██████████| 18/18 [00:24<00:00,  1.38s/it]


Epochs: 9 | Train Loss: 0.00029485346811540715             | Train Accuracy: 0.973624641254699

Saved the best model with accuracy: 0.973624641254699 to ./models/lstm.pth



100%|██████████| 18/18 [00:23<00:00,  1.33s/it]

Epochs: 10 | Train Loss: 0.0002766383837505996             | Train Accuracy: 0.9760095395933546

Saved the best model with accuracy: 0.9760095395933546 to ./models/lstm.pth

